In [ ]:
from vatic.engines import Simulator
from vatic.data.loaders import load_input, RtsLoader, T7kLoader
from vatic.engines import Simulator
from pgrisk.cost_attributor import CostAttributor

import pandas as pd
import numpy as np
from functools import reduce

from pathlib import Path
import dill as pickle
import os
from datetime import datetime
import matplotlib.pyplot as plt

RUC_MIPGAPS = {'RTS-GMLC': 0.01}
SCED_HORIZONS = {'RTS-GMLC': 4}

## Run unit commitment simulation

In [ ]:
grid = 'RTS-GMLC'  
num_days = 1  
start_date = '2020-01-15'
init_state_file = None
template, gen_data, load_data = load_input(grid, start_date, 
    num_days=num_days, init_state_file=init_state_file)

data_dir = Path('data', start_date)
os.makedirs(data_dir, exist_ok=True)
ruc_out = Path(data_dir, 'rts-gmlc-ruc.p')

siml = Simulator(template, gen_data, load_data, None,
                pd.to_datetime(start_date).date(), 1, solver='gurobi',
                solver_options={}, run_lmps=False, mipgap=RUC_MIPGAPS[grid],
                load_shed_penalty = 1e4, reserve_shortfall_penalty = 1e3,
                reserve_factor=0.05, output_detail=3,
                prescient_sced_forecasts=True, ruc_prescience_hour=0,
                ruc_execution_hour=16, ruc_every_hours=24,
                ruc_horizon=48, sced_horizon=SCED_HORIZONS[grid],
                lmp_shortfall_costs=False,
                enforce_sced_shutdown_ramprate=False,
                no_startup_shutdown_curves=False,
                init_ruc_file=ruc_out, verbosity=0,
                output_max_decimals=4, create_plots=False,
                renew_costs=None, save_to_csv=False, 
                last_conditions_file=None,)
_ = siml.simulate()


## Compute cost attribution between baseline and target
#### baseline uses day-ahead forecast as inputs
#### target uses the actual data as inputs

In [ ]:
baseline_gen_data = gen_data.copy(deep=True)
baseline_load_data = load_data.copy(deep=True)

for gen in baseline_gen_data.columns.get_level_values(1).unique():
    baseline_gen_data['actl', gen] = baseline_gen_data['fcst', gen]
for bus in baseline_load_data.columns.get_level_values(1).unique():
    baseline_load_data['actl', bus] = baseline_load_data['fcst', bus]

target_gen_data = gen_data.copy(deep=True)
target_load_data = load_data.copy(deep=True)

The cost attribution (code block below) can run at most 64 SCEDs and take long to finish.

In [ ]:
workdir = Path(data_dir, 'working')
renewable_gens = list(template['ForecastRenewables'])
load_buses = template['Buses']

attr = CostAttributor(template_data = template,
                start_date = pd.to_datetime(start_date), mipgap = 0.01,
                reserve_factor = 0.05, lmp_shortfall_costs = False,
                init_ruc_file = ruc_out, verbosity = 2,
                renewable_gens = renewable_gens, load_buses = load_buses,
                baseline_gen_data = baseline_gen_data, 
                baseline_load_data = baseline_load_data, 
                target_gen_data = target_gen_data, 
                target_load_data = target_load_data,
                load_shed_penalty = 1e3, reserve_shortfall_penalty = 0.,
                workdir = Path(workdir), scale = 6, tol = 0.1,)
attr.run_shadow_price_analysis()
attr.compute_attribution()

cost attribution of renewable generator "309_WIND_1" and bus "Abel".

In [ ]:
attr.cost_attribution['309_WIND_1'].head()

In [ ]:
attr.cost_attribution['Abel'].head()

Attribution summary

In [ ]:
attr.attribution_summary